In [114]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as functional
from torch.utils.data import Dataset, DataLoader
from skimage import io
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np


In [115]:
data_dir = "/Users/Tad/Documents/faceoffs"
data_initial = pd.read_csv("training_data_all_offensive_offensive.csv")
print(data_initial.shape)

(92904, 536)


In [116]:
data_no_na = data_initial.dropna() # data should already have no NAs due to numerical imputation
data = data_no_na.select_dtypes(['number'])
x = data.loc[:, data.columns != 'net_xg']
x = x.loc[:, x.columns != 'net_xg']
y = data['net_xg']

X_train, X_intermediate, y_train, y_intermediate = train_test_split(x, y, train_size = 0.6, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_intermediate, y_intermediate, train_size = 0.5, test_size=0.5, random_state=45)

# Above two lines in unison accomplish a 60-20-20 split for train-validation-test

In [117]:
device_selection = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_selection)
print(device_selection)
print(device)

if torch.cuda.is_available():
        torch.cuda.empty_cache()

train_batch_size = 1000
val_batch_size = 1000
test_batch_size = 1000

cuda
cuda


In [118]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, df):
        self.df = df
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        x = torch.tensor(self.df.iloc[idx, :-1].values, dtype=torch.float32)
        y = torch.tensor(self.df.iloc[idx, -1:].values, dtype=torch.float32)
        return x, y

train_df =  pd.concat([X_train, y_train], axis=1)  # pandas DataFrame containing training data
valid_df = pd.concat([X_val, y_val], axis=1) # pandas DataFrame containing validation data
test_df = pd.concat([X_test, y_test], axis=1)  # pandas DataFrame containing test data
train_dataset = CustomDataset(train_df)
valid_dataset = CustomDataset(valid_df)
test_dataset = CustomDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=val_batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

# Credit to ChatGPT for generating this cell of code in response to the following prompt: design a dataloader that accepts the output of a pandas train-validation-test split. Basically write code for a PyTorch dataloader that accepts pandas dataframes as arguments

In [119]:
class VanillaNeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_rate):
        super(VanillaNeuralNet, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout_rate = dropout_rate
        
        self.fc1 = nn.Linear(self.input_size, self.hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=self.dropout_rate)
        self.batch_norm = nn.BatchNorm1d(self.hidden_size)

        self.fc2 = nn.Linear(self.hidden_size, self.hidden_size)

        self.fc3 = nn.Linear(self.hidden_size, self.hidden_size)

        self.fc4 = nn.Linear(self.hidden_size, self.hidden_size)

        self.fc5 = nn.Linear(self.hidden_size, self.hidden_size)

        self.fc6 = nn.Linear(self.hidden_size, 1)

    def forward(self, x):

        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm(x)

        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm(x)

        x = self.fc3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm(x)

        x = self.fc4(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm(x)

        x = self.fc5(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm(x)

        x = self.fc6(x)
        
        return x


In [120]:
import optuna

num_epochs = 20
trial_count = 30

learning_rate = 0.01
loss_func = nn.MSELoss()

def train(model, train_loader, valid_loader, loss_func, optimizer, scheduler, num_epochs, device):
    training_losses = np.array([])
    validation_losses = np.array([])
    for epoch in range(num_epochs):
        print("\nEPOCH " +str(epoch+1)+" of "+str(num_epochs)+"\n")
        ########################### Training #####################################
        model.train(True)
        train_loss = 0
        count = 0
        for batch, (X, y) in enumerate(train_loader):
            X = X.float().to(device)
            y = y.float().to(device)
            optimizer.zero_grad()
            prediction = model(X).to(device)
            loss = loss_func(prediction, y.view(-1, 1))
            loss.backward()
            optimizer.step()
            scheduler.step()
            train_loss += loss.float().item()
            count += train_batch_size
        training_losses = np.append(training_losses, train_loss / count)
        print("Training loss:", train_loss / count)
        #model.train(False)

        ########################### Validation #####################################
        model.eval()
        val_loss = 0
        count = 0
        with torch.no_grad():
            for (X, y) in valid_loader:
                X = X.to(device)
                y = y.to(device)
                prediction = model(X).to(device)
                loss = loss_func(prediction, y.view(-1, 1))
                val_loss += loss.float().item()
                count += val_batch_size
        validation_losses = np.append(validation_losses, val_loss / count)
        print("Validation loss:", val_loss / count)
    return validation_losses[-1]

def objective(trial):
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    hidden_size = trial.suggest_int("hidden_size", 300, 600)
    dropout_rate = trial.suggest_float("dropout_rate", 0.2, 0.5)
    print("learning rate:", lr)
    print("weight_decay:", weight_decay)
    print("hidden_size:", hidden_size)
    print("dropout_rate:", dropout_rate)

    model = VanillaNeuralNet(input_dim, hidden_size, dropout_rate).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    return train(model, train_loader, valid_loader, loss_func, optimizer, scheduler, num_epochs, device)

study = optuna.create_study(direction="minimize")
input_dim = 529
output_dim = 1
hidden_dim = 300
dropout = 0.1
model = VanillaNeuralNet(input_dim, hidden_dim, dropout)
study.optimize(objective, n_trials=trial_count)
print(study.best_params)

[I 2023-04-24 08:49:13,036] A new study created in memory with name: no-name-479378b7-d94e-482b-b898-b02b6fc41af9


learning rate: 2.0763177318658052e-05
weight_decay: 5.8749109049964504e-06
hidden_size: 424
dropout_rate: 0.2211438696472659

EPOCH 1 of 20



[W 2023-04-24 08:49:17,677] Trial 0 failed with parameters: {'lr': 2.0763177318658052e-05, 'weight_decay': 5.8749109049964504e-06, 'hidden_size': 424, 'dropout_rate': 0.2211438696472659} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_1502/3036973307.py", line 63, in objective
    return train(model, train_loader, valid_loader, loss_func, optimizer, scheduler, num_epochs, device)
  File "/tmp/ipykernel_1502/3036973307.py", line 18, in train
    for batch, (X, y) in enumerate(train_loader):
  File "/home/ubuntu/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 634, in __next__
    data = self._next_data()
  File "/home/ubuntu/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 678, in _next_data
    data = self._dataset_fetcher.fetch(index

KeyboardInterrupt: 

In [ ]:
best_params = study.best_trial.params
print(best_params)
model = VanillaNeuralNet(input_dim, best_params["hidden_size"], best_params["dropout_rate"]).to(device)
optimizer = optim.Adam(model.parameters(), lr=best_params["lr"], weight_decay=best_params["weight_decay"])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
loss_func = nn.MSELoss()

num_epochs = 100 # training for more epochs on best model (which will be saved)

# Train the model with the best hyperparameters on the entire dataset
for epoch in range(num_epochs):
    ########################### Training #####################################
    model.train(True)
    train_loss = 0
    count = 0
    for batch, (X, y) in enumerate(train_loader):
        X = X.float().to(device)
        y = y.float().to(device)
        prediction = model(X).to(device)
        optimizer.zero_grad()
        loss = loss_func(prediction, y.view(-1, 1).float().to(device))
        loss.backward()
        optimizer.step()
        train_loss += loss.float().item()
        count += 1
    scheduler.step()
    print("Epoch", epoch+1, "Training loss:", train_loss / count)

# Save the best model
torch.save(model.state_dict(), "faceoffs_best_model_params.pt")
# save the trained model
torch.save(model, "faceoffs_best_model.pt")

{'lr': 0.0015164391793775465, 'weight_decay': 0.0006662013727345294, 'hidden_size': 376, 'dropout_rate': 0.26792380977585883}
Epoch 1 Training loss: 0.09280126615028296
Epoch 2 Training loss: 0.010001694862564494
Epoch 3 Training loss: 0.003176360800612851
Epoch 4 Training loss: 0.0018281558940153836
Epoch 5 Training loss: 0.001573180571930217
Epoch 6 Training loss: 0.0015318818436104006
Epoch 7 Training loss: 0.0015313260747851537
Epoch 8 Training loss: 0.0015245866421277501
Epoch 9 Training loss: 0.0013323610588226334
Epoch 10 Training loss: 0.0004332387914993368


In [ ]:
test_loss = 0
y_true = []
y_pred = []

model.eval()
with torch.no_grad():
    for X, y in test_loader:
        X = X.to(device)
        y = y.to(device)
        prediction = model(X).to(device)
        # Remove extra dimensions from y
        y = y.unsqueeze(1).squeeze()
        loss = loss_func(prediction.squeeze(), y)
        test_loss += loss.float().item()
        y_true.extend(y.cpu().numpy())
        y_pred.extend(prediction.cpu().numpy().squeeze())

test_loss /= len(test_loader)
print(f"Test Loss: {test_loss:.4f}")

from sklearn.metrics import r2_score
r2 = r2_score(y_true, y_pred)
print("True labels:", y_true)
print("Predictions:", y_pred)
print(f"R-squared Coefficient: {r2:.4f}")


Test Loss: 0.0015
True labels: [0.00920247, 0.0160865, 0.00920247, 0.0109509, 0.0, 0.0, 0.0115287, 0.00920247, 0.0, 0.029275, 0.0100969, 0.0, 0.0166409, 0.00927501, 0.00920247, 0.0, 0.00920247, 0.0, 0.00920247, 0.00920247, 0.00920247, 0.013338, 0.00920247, 0.00920247, 0.0, 0.00920247, 0.00920247, 0.150549, 0.00920247, 0.00920247, 0.00920247, 0.00920247, 0.0, 0.0, 0.00920247, 0.00920247, 0.00920247, 0.00920247, 0.0, 0.0194755, 0.0860846, 0.0287319, 0.0107397, 0.00920247, 0.00920247, 0.00920247, 0.0, 0.00920247, 0.0, 0.0188868, 0.00920247, 0.0856539, 0.00920247, 0.00920247, 0.192434, 0.00920247, 0.0405562, 0.00920247, 0.00925497, 0.00920247, 0.00920247, 0.0121461, 0.0676457, 0.0390529, 0.00920247, 0.00920247, 0.0941837, 0.00920247, 0.0376101, 0.0584831, 0.0751837, 0.00920247, 0.0457924, 0.00920247, 0.00920247, 0.00920247, 0.00920247, 0.0600364, 0.00920247, 0.00920247, 0.00920247, 0.00920247, 0.00920247, 0.026531, 0.00920247, 0.00920247, 0.0611948, 0.00920247, 0.00920247, 0.00920247, 0.0,